In [133]:
#First import data and filter to the 'text' column
import pandas as pd

#all years
data = pd.read_json('test.json')
df = pd.DataFrame(data)

##Data cleaning
df['text'] = df['text'].astype(str).replace(r'\\n',' ', regex=True)
df['text'] = df['text'].astype(str).replace(r'<','', regex=True)
df['text'] = df['text'].astype(str).replace(r'>','', regex=True)
df['text'] = df['text'].astype(str).replace(r'. ',' ', regex=True)
df['text'] = df['text'].astype(str).replace(r'\[','', regex=True)
df['text'] = df['text'].astype(str).replace(r'\]','', regex=True)
df['text'] = df['text'].astype(str).replace(r'\'','', regex=True)

textsAll = df['text']

#young years
youth = "1869, 1870, 1871, 1872, 1873, 1874, 1875"
df_youth = df[df['year']<=1875]


#turning point
turningP = "1876, 1877, 1878"


#maturity
mature = "1879, 1880, 1881, 1882, 1883, 1884, 1885, 1886, 1887, 1888"
df_mature = df[df['year']>=1879]

df_mature.to_csv('df_mature.csv')


In [135]:
documents = data['text']

from pprint import pprint #pretty-printer
from collections import defaultdict
from nltk.corpus import stopwords

#remove common words
stoplist = stopwords.words('german')
#additional stopwords
stopwords_new = ['.', 'ii.']
stoplist.append(stopwords_new) #if I need to add more stopwords manually

#tokenize
texts = [
    [word for word in document.lower().split() if word not in stoplist] 
    for document in documents
]

#remove words that appear only once

frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

In [136]:
from gensim import corpora

dictionary = corpora.Dictionary(texts)
dictionary.save('/tmp/document1.dict')

# dictionary.token2id


In [141]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

from gensim.test.utils import get_tmpfile
fname = get_tmpfile("my_doc2vec_model")

model.save(fname)
model = Doc2Vec.load(fname)

model
# common_texts = [] # initialise empty list
# for ind, row in df.iteritem():
#     text = row[name_column_with_text]
#     common_texts.append(text)


# common_dictionary = Dictionary(common_texts)
# common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]

# lda = LdaModel(common_corpus, num_topics=10)

# model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
# model.build_vocab(dictionary)